In [12]:
import plotly.express as px
import pandas as pd
import numpy as np
from Project.Database import Db
from Project._04TPMAlgorithm.transform_for_TPM_algorithm import create_column_translator

In [13]:
def find_status_one(dataframe, status_atts):
    if isinstance(status_atts, list) and len(status_atts) >= 1:
        return dataframe.loc[(lambda self: self[status_atts].sum(1) == len(status_atts))]
    elif isinstance(status_atts, str):
        return dataframe.loc[(lambda self: self[status_atts] == 1)]
    else:
        return dataframe

In [14]:
def find_status_zero(dataframe, status_atts):
    if isinstance(status_atts, list) and len(status_atts) >= 1:
        return dataframe.loc[(lambda self: self[status_atts].sum(1) == 0)]
    elif isinstance(status_atts, str):
        return dataframe.loc[(lambda self: self[status_atts] == 0)]
    else:
        return dataframe

In [15]:
def find_status_ppl_same_floor(status_row):
    tmp_meta = status_cols.copy()
    return tmp_meta.loc[(lambda self: (self.index.str.contains('SensHeat')) & (
            self['Measurement_Floor'] == status_row['Measurement_Floor']))].index.tolist()

In [16]:
def find_status_siblings(status_row, status_att):
    tmp_meta = status_cols.copy()
    tmp_meta.drop(status_att, inplace=True)
    return tmp_meta.loc[(lambda self: (self['Load_Match'] == status_row['Load_Match']))].index.tolist()

In [17]:
def find_status_siblings_zero(dataframe, status_row, status_att):
    return find_status_zero(dataframe, find_status_siblings(status_row, status_att))

In [18]:
def remove_zero_consumption(dataframe, status_row):
    return dataframe.loc[lambda self: self[status_row['Load_Match']] > 0]

In [19]:
def find_power_consumption(dataframe, status_row, status_att):
    dataframe = remove_zero_consumption(find_status_one(dataframe, status_att), status_row)
    # return dataframe[status_row['Load_Match']].div(dataframe[find_status_siblings(status_row, status_att)].sum(1) + 1).mean()
    dataframe_siblings_zero = find_status_siblings_zero(dataframe, status_row, status_att)
    if dataframe_siblings_zero.shape[0] > 0:
        return dataframe_siblings_zero[status_row['Load_Match']].mean()
    else:
        return dataframe[status_row['Load_Match']].div(
            dataframe[find_status_siblings(status_row, status_att)].sum(1) + 1).mean()

In [20]:
def find_redundancy(house, status_cols, energy=False):
    redundancy_df = pd.DataFrame(columns=['RedundantMinutes', 'PowerConsumption', 'EnergyConsumed'])
    timestamp_dict = {}

    appliance_status = status_cols.loc[(lambda self:
                                        (~self.index.str.contains('SensHeat')) &
                                        (~self['Measurement_Location'].isin(['Kitchen', 'Utility'])) &
                                        (self['Subsystem'] == 'Loads'))]

    for att, row in appliance_status.iterrows():

        power_consumption = find_power_consumption(house, row, att)
        tmp_df = find_status_one(house, att)

        if att == 'Load_StatusPlugLoadVacuum':
            ppl = status_cols.loc[(lambda self: (self.index.str.contains('SensHeat')))].index.tolist()

        elif row['Measurement_Location'] == 'Bedroom2':
            ppl = 'Load_StatusSensHeatChildAUP'

        elif row['Measurement_Location'] == 'Bedroom3':
            ppl = 'Load_StatusSensHeatChildBUP'

        elif row['Measurement_Location'] in ['MasterBedroom', 'MasterBathroom']:
            ppl = ['Load_StatusSensHeatPrntAUP', 'Load_StatusSensHeatPrntBUP']

        elif row['Measurement_Location'] == 'Bedroom4':
            ppl = ['Load_StatusSensHeatPrntADOWN', 'Load_StatusSensHeatPrntBDOWN']

        else:
            ppl = find_status_ppl_same_floor(row)

        tmp_df = find_status_zero(tmp_df, ppl)

        redundancy_df.loc[att] = {
            'PowerConsumption': power_consumption,
            'RedundantMinutes': tmp_df.shape[0],
            'EnergyConsumed': f'{tmp_df.shape[0] * power_consumption / 60 / 1000} kWh'
        }
        timestamp_dict[att] = tmp_df['Timestamp']

    return redundancy_df, timestamp_dict

In [24]:
year_redundancy_df_dict = {}
timestamp_dict = {}
for year in [1, 2]:
    NZERTF, NZERTF_meta = Db.load_data(hourly=False, meta=True, year=year)
    status_cols = NZERTF_meta.loc[(lambda self: self['Units'] == 'BinaryStatus')]

    year_redundancy_df_dict[f'Year{year}'], timestamp_dict[f'Year{year}'] = find_redundancy(NZERTF, status_cols)

year_redundancy_df_dict, timestamp_dict

({'Year1':                                       RedundantMinutes  PowerConsumption  \
  Load_StatusBA1Lights                               215         42.648803   
  Load_StatusBA2Lights                              1515         11.609812   
  Load_StatusBR2Lights                             12081         40.063462   
  Load_StatusBR3Lights                             11651         33.199784   
  Load_StatusBR4Lights                                60         46.461190   
  Load_StatusDRLights                                  1         43.319580   
  Load_StatusEntryHallLights                           1         20.188646   
  Load_StatusLRLights1                                 0         22.774127   
  Load_StatusLRLights2                               410         27.784904   
  Load_StatusLRLights3                                 0         27.696870   
  Load_StatusMBALights                               117         84.547482   
  Load_StatusMBRLights1                               7

In [25]:
for year in [1, 2]:
    year_redundancy_df_dict[f'Year{year}']['PowerConsumption'] = pd.DataFrame(year_redundancy_df_dict[f'Year{year}']['PowerConsumption']).merge(right=year_redundancy_df_dict[f'Year{2 if year == 1 else 1}']['PowerConsumption'], how='left', left_index=True, right_index=True).mean(1)
    year_redundancy_df_dict[f'Year{year}']['EnergyConsumed'] = year_redundancy_df_dict[f"Year{year}"]["RedundantMinutes"].multiply(year_redundancy_df_dict[f"Year{year}"]["PowerConsumption"]).div(60).div(1000)

year_redundancy_df_dict

{'Year1':                                       RedundantMinutes  PowerConsumption  \
 Load_StatusBA1Lights                               215         43.418675   
 Load_StatusBA2Lights                              1515         11.471449   
 Load_StatusBR2Lights                             12081         40.012476   
 Load_StatusBR3Lights                             11651         33.193948   
 Load_StatusBR4Lights                                60         46.461190   
 Load_StatusDRLights                                  1         43.162324   
 Load_StatusEntryHallLights                           1         20.188646   
 Load_StatusLRLights1                                 0         22.774127   
 Load_StatusLRLights2                               410         27.758224   
 Load_StatusLRLights3                                 0         27.714207   
 Load_StatusMBALights                               117         84.493207   
 Load_StatusMBRLights1                               75         24.

In [26]:
total_minutes = 0
total_energy = 0

for year in [1,2]:
    total_minutes += year_redundancy_df_dict[f'Year{year}']['RedundantMinutes'].sum() / 60
    total_energy += year_redundancy_df_dict[f'Year{year}']['EnergyConsumed'].sum()

total_minutes = str(round(total_minutes, 2)) + ' hours'
total_energy = str(round(total_energy, 4)) + ' kWh'
total_minutes, total_energy

('2028.52 hours', '119.4787 kWh')

In [27]:
status_cols.loc[lambda self: self['Measurement_Location'] == 'Kitchen', 'Load_Match']

Unnamed: 0
Load_StatusKitchenLightsA                Load_1stFloorLightsPowerUsage
Load_StatusKitchenLightsB                Load_1stFloorLightsPowerUsage
Load_StatusKitchenLightsC                Load_1stFloorLightsPowerUsage
Load_StatusSensHeatPrntADOWN         Elec_PowerHeatLoadforRefrigerator
Load_StatusPlugLoadBlender                   Load_KPlugLoadsPowerUsage
Load_StatusPlugLoadToasterOven               Load_KPlugLoadsPowerUsage
Load_StatusApplianceRangeHood                                      NaN
Load_StatusApplianceCooktop                        Load_OvenPowerTotal
Load_StatusApplianceOven                           Load_OvenPowerTotal
Load_StatusPlugLoadSlowCooker                Load_KPlugLoadsPowerUsage
Load_StatusPlugLoadToaster                   Load_KPlugLoadsPowerUsage
Load_StatusApplianceDishwasher                                     NaN
Load_StatusPlugLoadHandMixer                 Load_KPlugLoadsPowerUsage
Load_StatusPlugLoadCanOpener                 Load_KPlugLoadsPowerU

In [37]:
timestamp_dict['Year1']['Load_StatusBA1Lights'].groupby(timestamp_dict['Year1']['Load_StatusBA1Lights'].dt.hour).size()

Timestamp
20     39
21    118
22     58
Name: Timestamp, dtype: int64

## Calculate the watt used in redudant intervals

## Plot the redundant time intervals and highlight them with colors

In [ ]:
fig = px.scatter(NZERTF, x='Timestamp', y='Load_MBRPlugLoadsPowerUsage')
fig.show()